# Chapter 14: Penalized Regression
## Practice Activities

This notebook contains all practice activities from Chapter 14 of "Data Science and Machine Learning with Python."

**Topics Covered:**
- Ridge Regression
- LASSO (Least Absolute Shrinkage and Selection Operator)
- Elastic Net
- Hyperparameter Tuning
- Coefficient Comparison

---

## 📦 Setup: Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Scikit-learn components
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ All libraries imported successfully!")

## 📊 Load and Clean the AMES Housing Dataset

Following the chapter, we'll:
1. Load the dataset
2. Remove columns with too many NaN values
3. Drop remaining NaN values
4. Prepare features and target variable

In [ ]:
# Load the dataset
ames = pd.read_csv("AmesHousing.csv")

print(f"Original dataset shape: {ames.shape}")
print(f"\nFirst few rows:")
ames.head()

In [ ]:
# Get rid of columns with mostly NaN values (as shown in the chapter)
good_cols = ames.isna().sum() < 100
ames = ames.loc[:, good_cols]

# Drop other NAs
ames = ames.dropna()

print(f"✓ Cleaned dataset shape: {ames.shape}")
print(f"\nColumns remaining: {len(ames.columns)}")

## 🎯 Prepare Data for Modeling

Following the chapter example, we'll:
- Drop `SalePrice`, `Order`, and `PID` columns
- Set up X (features) and y (target)

In [ ]:
# Prepare features and target
X = ames.drop(["SalePrice", "Order", "PID"], axis=1)
y = ames["SalePrice"]

print(f"✓ Features (X) shape: {X.shape}")
print(f"✓ Target (y) shape: {y.shape}")
print(f"\nNumber of features: {X.shape[1]}")
print(f"Number of observations: {X.shape[0]}")

## 🔧 Create Preprocessing Pipeline

Following the chapter, we'll create a ColumnTransformer that:
- Dummifies categorical variables (with OneHotEncoder)
- Standardizes numeric variables (with StandardScaler)
- Uses `make_column_selector` to automatically identify variable types

In [ ]:
# Create the ColumnTransformer
ct = ColumnTransformer(
    [
        ("dummify",
         OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
         make_column_selector(dtype_include=object)),
        ("standardize",
         StandardScaler(),
         make_column_selector(dtype_include=np.number))
    ],
    remainder="passthrough"
)

print("✓ ColumnTransformer created successfully!")
print("\nTransformer steps:")
print("  1. OneHotEncoder for categorical variables (dtype=object)")
print("  2. StandardScaler for numeric variables (dtype=number)")

## 📐 Baseline: Ordinary Linear Regression

Before we try penalized regression, let's see how regular linear regression performs.

In [ ]:
# Create OLS pipeline
lr_pipeline = Pipeline([
    ("preprocessing", ct),
    ("linear_regression", LinearRegression())
])

# Cross-validate
print("Cross-validating Ordinary Linear Regression...")
lr_scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='r2')
lr_cv_r2 = lr_scores.mean()

print("\n" + "=" * 60)
print("BASELINE: Ordinary Linear Regression (OLS)")
print("=" * 60)
print(f"Cross-Validated R²: {lr_cv_r2:.4f}")
print(f"R² scores across folds: {lr_scores}")
print(f"Standard deviation: {lr_scores.std():.4f}")
print("=" * 60)

# Fit on full data to get coefficients
lr_pipeline.fit(X, y)
lr_coef = lr_pipeline.named_steps['linear_regression'].coef_

print(f"\nNumber of coefficients: {len(lr_coef)}")
print(f"\nThis will be our baseline for comparison!")

### Check for Overfitting

In [ ]:
# Check negative R² (sign of severe overfitting)
print("\n⚠ OVERFITTING CHECK:")
print("=" * 60)

if lr_cv_r2 < 0:
    print("❌ SEVERE OVERFITTING DETECTED!")
    print(f"   Cross-validated R² is NEGATIVE: {lr_cv_r2:.4f}")
    print("\n   This means the model performs WORSE than just predicting the mean!")
    print("   We used too many features and fit too closely to training data.")
    print("\n   ➡ This is why we need PENALIZED REGRESSION!")
elif lr_cv_r2 < 0.3:
    print("⚠ Poor performance detected.")
    print(f"   Cross-validated R² is low: {lr_cv_r2:.4f}")
    print("   The model may be overfitting or the features may not be informative enough.")
else:
    print("✓ Model performance is reasonable.")
    print(f"   Cross-validated R²: {lr_cv_r2:.4f}")
    print("   However, penalized regression may still improve generalization.")

print("=" * 60)

---

# 📝 Practice Activity 1: Ridge Regression

**Task (Section 14.2):** 
1. Make a pipeline that uses all the variables in the Ames dataset, and then fits Ridge Regression with λ = 1
2. Cross-validate this pipeline and compare the results to ordinary linear regression
3. Then fit the model on the whole dataset and get the coefficients
4. Make a plot of these coefficients compared to the ones from ordinary linear regression

## What is Ridge Regression?

Ridge Regression adds a **penalty** to the loss function:

$$\ell(\beta) = \sum_{i=1}^{n}(\hat{y}_i - y_i)^2 + \lambda \sum_{j=1}^{p}\beta_j^2$$

Where:
- First term: Sum of Squared Errors (SSE)
- Second term: **Ridge penalty** (sum of squared coefficients)
- λ: Hyperparameter controlling the penalty strength

**Effect:** Ridge makes coefficients **smaller** (shrinks them toward zero), reducing overfitting.

## Step 1: Ridge Regression with λ = 1

In [ ]:
# Create Ridge pipeline with lambda = 1
# Note: sklearn uses 'alpha' for the parameter we call lambda
ridge_pipeline_1 = Pipeline([
    ("preprocessing", ct),
    ("ridge_regression", Ridge(alpha=1))
])

# Cross-validate
print("Cross-validating Ridge Regression (λ = 1)...")
ridge_scores_1 = cross_val_score(ridge_pipeline_1, X, y, cv=5, scoring='r2')
ridge_cv_r2_1 = ridge_scores_1.mean()

print("\n" + "=" * 60)
print("RIDGE REGRESSION (λ = 1)")
print("=" * 60)
print(f"Cross-Validated R²: {ridge_cv_r2_1:.4f}")
print(f"R² scores across folds: {ridge_scores_1}")
print(f"Standard deviation: {ridge_scores_1.std():.4f}")
print("=" * 60)

## Step 2: Compare to Ordinary Linear Regression

In [ ]:
# Create comparison
comparison_df = pd.DataFrame({
    'Model': ['Ordinary Linear Regression', 'Ridge Regression (λ=1)'],
    'Cross-Validated R²': [lr_cv_r2, ridge_cv_r2_1],
    'Improvement': ['Baseline', f"{(ridge_cv_r2_1 - lr_cv_r2):.4f}"]
})

print("\n" + "=" * 70)
print("COMPARISON: OLS vs Ridge (λ=1)")
print("=" * 70)
print(comparison_df.to_string(index=False))
print("\n" + "=" * 70)

if ridge_cv_r2_1 > lr_cv_r2:
    improvement = ((ridge_cv_r2_1 - lr_cv_r2) / abs(lr_cv_r2)) * 100
    print(f"\n✓ Ridge Regression IMPROVED performance!")
    print(f"  Improvement: {ridge_cv_r2_1 - lr_cv_r2:.4f} ({improvement:.1f}% better)")
    print(f"\n💡 The penalty helped reduce overfitting!")
else:
    print(f"\n⚠ Ridge did not improve over OLS with λ=1")
    print(f"  This suggests we may need to tune λ to find the optimal value.")

print("=" * 70)

## Step 3: Fit on Full Dataset and Get Coefficients

In [ ]:
# Fit Ridge on full dataset
ridge_pipeline_1.fit(X, y)
ridge_coef_1 = ridge_pipeline_1.named_steps['ridge_regression'].coef_

print(f"✓ Ridge Regression fitted on full dataset")
print(f"\nNumber of coefficients: {len(ridge_coef_1)}")
print(f"\nCoefficient statistics:")
print(f"  Mean: {ridge_coef_1.mean():.2f}")
print(f"  Std: {ridge_coef_1.std():.2f}")
print(f"  Min: {ridge_coef_1.min():.2f}")
print(f"  Max: {ridge_coef_1.max():.2f}")

## Step 4: Plot Coefficients Comparison

In [ ]:
# Create coefficient comparison plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Scatter plot comparison
axes[0].scatter(lr_coef, ridge_coef_1, alpha=0.5, s=30)
axes[0].plot([lr_coef.min(), lr_coef.max()], 
             [lr_coef.min(), lr_coef.max()], 
             'r--', linewidth=2, label='y=x (no shrinkage)')
axes[0].set_xlabel('OLS Coefficients', fontsize=12)
axes[0].set_ylabel('Ridge Coefficients (λ=1)', fontsize=12)
axes[0].set_title('Coefficient Comparison: OLS vs Ridge\n(Points below red line show shrinkage)', 
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Histogram of absolute values
axes[1].hist(np.abs(lr_coef), bins=50, alpha=0.5, label='OLS', color='blue')
axes[1].hist(np.abs(ridge_coef_1), bins=50, alpha=0.5, label='Ridge (λ=1)', color='red')
axes[1].set_xlabel('Absolute Coefficient Value', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of Coefficient Magnitudes\n(Ridge coefficients are generally smaller)', 
                  fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate shrinkage statistics
shrinkage_pct = np.mean(np.abs(ridge_coef_1) < np.abs(lr_coef)) * 100
print(f"\n📊 COEFFICIENT SHRINKAGE:")
print(f"  {shrinkage_pct:.1f}% of Ridge coefficients are smaller in magnitude than OLS")
print(f"\n  Average absolute OLS coefficient: {np.abs(lr_coef).mean():.2f}")
print(f"  Average absolute Ridge coefficient: {np.abs(ridge_coef_1).mean():.2f}")
print(f"  Reduction: {(1 - np.abs(ridge_coef_1).mean()/np.abs(lr_coef).mean())*100:.1f}%")

---

# 📝 Practice Activity 2: Ridge Tuning

**Task (Section 14.2.1):**
Using the same pipeline as previously, perform tuning on λ.

You should always try λ values on a **log scale**; that is, don't use `[1,2,3,4]`; instead use something like `[0.001, 0.01, 0.1, 1, 10]`

## Why Log Scale?

λ controls penalty strength:
- λ = 0: No penalty (OLS)
- λ = 0.001: Very small penalty
- λ = 1: Moderate penalty
- λ = 100: Strong penalty (coefficients → 0)

Log scale lets us explore a wide range efficiently!

In [ ]:
# Create Ridge pipeline for tuning
ridge_pipeline = Pipeline([
    ("preprocessing", ct),
    ("ridge_regression", Ridge())
])

# Define parameter grid on log scale
param_grid_ridge = {
    'ridge_regression__alpha': [0.001, 0.01, 0.1, 1, 10]
}

print("Setting up GridSearchCV for Ridge Regression...")
print(f"Testing λ values: {param_grid_ridge['ridge_regression__alpha']}")
print("Using 5-fold cross-validation\n")

# Perform grid search
ridge_gscv = GridSearchCV(
    ridge_pipeline,
    param_grid_ridge,
    cv=5,
    scoring='r2',
    return_train_score=True,
    verbose=1
)

print("\nFitting GridSearchCV (this may take a moment)...")
ridge_gscv.fit(X, y)
print("\n✓ Grid search complete!")

## Ridge Tuning Results

In [ ]:
# Extract results
ridge_cv_results = pd.DataFrame(ridge_gscv.cv_results_)
ridge_results_summary = pd.DataFrame({
    'Lambda (λ)': param_grid_ridge['ridge_regression__alpha'],
    'Mean CV R²': ridge_cv_results['mean_test_score'],
    'Std CV R²': ridge_cv_results['std_test_score'],
    'Mean Train R²': ridge_cv_results['mean_train_score']
})

print("\n" + "=" * 70)
print("RIDGE TUNING RESULTS")
print("=" * 70)
print(ridge_results_summary.to_string(index=False))
print("\n" + "=" * 70)
print(f"\n✓ Best λ: {ridge_gscv.best_params_['ridge_regression__alpha']}")
print(f"  Best CV R²: {ridge_gscv.best_score_:.4f}")
print("=" * 70)

## Visualization: λ vs Performance

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Performance vs Lambda
lambdas = ridge_results_summary['Lambda (λ)']
axes[0].semilogx(lambdas, ridge_results_summary['Mean Train R²'], 
                 marker='o', label='Training R²', linewidth=2, markersize=8)
axes[0].semilogx(lambdas, ridge_results_summary['Mean CV R²'], 
                 marker='s', label='Cross-Validation R²', linewidth=2, markersize=8)
axes[0].axvline(x=ridge_gscv.best_params_['ridge_regression__alpha'], 
                color='red', linestyle='--', alpha=0.7, label='Best λ')
axes[0].set_xlabel('Lambda (λ) - Log Scale', fontsize=12)
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Ridge Regression: Performance vs Penalty Strength\n(Higher is Better)', 
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Overfitting gap
gap = ridge_results_summary['Mean Train R²'] - ridge_results_summary['Mean CV R²']
axes[1].semilogx(lambdas, gap, marker='o', linewidth=2, markersize=8, color='purple')
axes[1].axvline(x=ridge_gscv.best_params_['ridge_regression__alpha'], 
                color='red', linestyle='--', alpha=0.7, label='Best λ')
axes[1].axhline(y=0.05, color='orange', linestyle='--', alpha=0.7, 
                label='Overfitting Threshold')
axes[1].set_xlabel('Lambda (λ) - Log Scale', fontsize=12)
axes[1].set_ylabel('Train R² - CV R² (Gap)', fontsize=12)
axes[1].set_title('Overfitting vs Penalty Strength\n(Lower Gap is Better)', 
                  fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 KEY OBSERVATIONS:")
print(f"  • As λ increases, penalty gets stronger")
print(f"  • Optimal λ = {ridge_gscv.best_params_['ridge_regression__alpha']} balances bias and variance")
print(f"  • Training R² decreases with larger λ (more regularization)")
print(f"  • CV R² peaks at optimal λ (best generalization)")

---

# 📝 Practice Activity 3: LASSO Regression

**Task (Section 14.3.1):**
1. Create a LASSO pipeline, and tune λ
2. Fit your best model on the full Ames data, and compare the coefficients to Ridge and OLS

## What is LASSO?

LASSO (Least Absolute Shrinkage and Selection Operator) uses **absolute value** penalty:

$$\ell(\beta) = \sum_{i=1}^{n}(\hat{y}_i - y_i)^2 + \lambda \sum_{j=1}^{p}|\beta_j|$$

**Key Difference from Ridge:**
- Ridge: Shrinks coefficients toward zero (but rarely exactly zero)
- LASSO: Can make coefficients **exactly zero** (automatic feature selection!)

**Effect:** LASSO performs **automatic feature selection** by eliminating unimportant predictors.

In [ ]:
# Create LASSO pipeline
lasso_pipeline = Pipeline([
    ("preprocessing", ct),
    ("lasso_regression", Lasso())
])

# Define parameter grid on log scale
param_grid_lasso = {
    'lasso_regression__alpha': [0.001, 0.01, 0.1, 1, 10]
}

print("Setting up GridSearchCV for LASSO Regression...")
print(f"Testing λ values: {param_grid_lasso['lasso_regression__alpha']}")
print("Using 5-fold cross-validation\n")

# Perform grid search
lasso_gscv = GridSearchCV(
    lasso_pipeline,
    param_grid_lasso,
    cv=5,
    scoring='r2',
    return_train_score=True,
    verbose=1
)

print("\nFitting GridSearchCV (this may take a moment)...")
lasso_gscv.fit(X, y)
print("\n✓ Grid search complete!")

## LASSO Tuning Results

In [ ]:
# Extract results
lasso_cv_results = pd.DataFrame(lasso_gscv.cv_results_)
lasso_results_summary = pd.DataFrame({
    'Lambda (λ)': param_grid_lasso['lasso_regression__alpha'],
    'Mean CV R²': lasso_cv_results['mean_test_score'],
    'Std CV R²': lasso_cv_results['std_test_score'],
    'Mean Train R²': lasso_cv_results['mean_train_score']
})

print("\n" + "=" * 70)
print("LASSO TUNING RESULTS")
print("=" * 70)
print(lasso_results_summary.to_string(index=False))
print("\n" + "=" * 70)
print(f"\n✓ Best λ: {lasso_gscv.best_params_['lasso_regression__alpha']}")
print(f"  Best CV R²: {lasso_gscv.best_score_:.4f}")
print("=" * 70)

# Fit best model on full data
best_lasso = lasso_gscv.best_estimator_
lasso_coef = best_lasso.named_steps['lasso_regression'].coef_

# Count zero coefficients (feature selection!)
n_zero_coef = np.sum(lasso_coef == 0)
n_nonzero_coef = np.sum(lasso_coef != 0)

print(f"\n🎯 FEATURE SELECTION:")
print(f"  Total features: {len(lasso_coef)}")
print(f"  Non-zero coefficients: {n_nonzero_coef}")
print(f"  Zero coefficients (eliminated): {n_zero_coef}")
print(f"  Features kept: {(n_nonzero_coef/len(lasso_coef))*100:.1f}%")
print(f"\n  💡 LASSO automatically eliminated {n_zero_coef} features!")

## Compare LASSO to Ridge and OLS

In [ ]:
# Get best Ridge model coefficients
best_ridge = ridge_gscv.best_estimator_
ridge_coef_best = best_ridge.named_steps['ridge_regression'].coef_

# Create comprehensive comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: OLS vs Ridge
axes[0, 0].scatter(lr_coef, ridge_coef_best, alpha=0.5, s=30, color='blue')
axes[0, 0].plot([lr_coef.min(), lr_coef.max()], 
                [lr_coef.min(), lr_coef.max()], 
                'r--', linewidth=2, label='No shrinkage')
axes[0, 0].set_xlabel('OLS Coefficients', fontsize=11)
axes[0, 0].set_ylabel('Ridge Coefficients', fontsize=11)
axes[0, 0].set_title('OLS vs Ridge\n(Ridge shrinks but rarely zeros)', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Plot 2: OLS vs LASSO
colors = ['red' if c == 0 else 'green' for c in lasso_coef]
axes[0, 1].scatter(lr_coef, lasso_coef, alpha=0.5, s=30, c=colors)
axes[0, 1].plot([lr_coef.min(), lr_coef.max()], 
                [lr_coef.min(), lr_coef.max()], 
                'r--', linewidth=2, label='No shrinkage')
axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
axes[0, 1].set_xlabel('OLS Coefficients', fontsize=11)
axes[0, 1].set_ylabel('LASSO Coefficients', fontsize=11)
axes[0, 1].set_title(f'OLS vs LASSO\n(Red = eliminated features: {n_zero_coef})', 
                     fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Plot 3: Ridge vs LASSO
colors = ['red' if c == 0 else 'purple' for c in lasso_coef]
axes[1, 0].scatter(ridge_coef_best, lasso_coef, alpha=0.5, s=30, c=colors)
axes[1, 0].plot([ridge_coef_best.min(), ridge_coef_best.max()], 
                [ridge_coef_best.min(), ridge_coef_best.max()], 
                'r--', linewidth=2, label='Equal coefficients')
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
axes[1, 0].set_xlabel('Ridge Coefficients', fontsize=11)
axes[1, 0].set_ylabel('LASSO Coefficients', fontsize=11)
axes[1, 0].set_title('Ridge vs LASSO\n(LASSO zeros out features Ridge keeps)', 
                     fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Plot 4: Distribution comparison
axes[1, 1].hist(np.abs(lr_coef), bins=50, alpha=0.4, label='OLS', color='blue')
axes[1, 1].hist(np.abs(ridge_coef_best), bins=50, alpha=0.4, label='Ridge', color='green')
axes[1, 1].hist(np.abs(lasso_coef[lasso_coef != 0]), bins=50, alpha=0.4, 
                label=f'LASSO (non-zero only)', color='red')
axes[1, 1].set_xlabel('Absolute Coefficient Value', fontsize=11)
axes[1, 1].set_ylabel('Frequency', fontsize=11)
axes[1, 1].set_title('Coefficient Magnitude Distribution', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print("\n" + "=" * 70)
print("COEFFICIENT STATISTICS")
print("=" * 70)
print(f"\nOLS:")
print(f"  Mean |coef|: {np.abs(lr_coef).mean():.4f}")
print(f"  Non-zero: {len(lr_coef)} (100%)")

print(f"\nRidge (λ={ridge_gscv.best_params_['ridge_regression__alpha']}):")
print(f"  Mean |coef|: {np.abs(ridge_coef_best).mean():.4f}")
print(f"  Non-zero: {np.sum(ridge_coef_best != 0)} ({np.sum(ridge_coef_best != 0)/len(ridge_coef_best)*100:.1f}%)")
print(f"  Shrinkage: {(1 - np.abs(ridge_coef_best).mean()/np.abs(lr_coef).mean())*100:.1f}%")

print(f"\nLASSO (λ={lasso_gscv.best_params_['lasso_regression__alpha']}):")
print(f"  Mean |coef| (non-zero): {np.abs(lasso_coef[lasso_coef != 0]).mean():.4f}")
print(f"  Non-zero: {n_nonzero_coef} ({(n_nonzero_coef/len(lasso_coef))*100:.1f}%)")
print(f"  Features eliminated: {n_zero_coef}")
print("=" * 70)

---

# 📝 Practice Activity 4: Elastic Net

**Task (Section 14.3.3):**
1. Create an Elastic Net pipeline, and tune λ and α
2. Fit your best model on the full Ames data, and compare the coefficients to Ridge and OLS

## What is Elastic Net?

Elastic Net combines **both** Ridge and LASSO penalties:

$$\ell(\beta) = SSE + \lambda \left(\alpha \sum_{j=1}^{p}|\beta_j| + (1-\alpha)\sum_{j=1}^{p}\beta_j^2\right)$$

Where:
- λ: Overall penalty strength
- α: Balance between L1 (LASSO) and L2 (Ridge)
  - α = 0: Pure Ridge
  - α = 1: Pure LASSO
  - α = 0.5: Equal mix

**Why use it?** Gets the **best of both worlds**:
- Ridge's stability
- LASSO's feature selection

In [ ]:
# Create Elastic Net pipeline
elasticnet_pipeline = Pipeline([
    ("preprocessing", ct),
    ("elasticnet_regression", ElasticNet())
])

# Define parameter grid - tune BOTH lambda and alpha
param_grid_elasticnet = {
    'elasticnet_regression__alpha': [0.001, 0.01, 0.1, 1, 10],  # This is our λ
    'elasticnet_regression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  # This is our α
}

print("Setting up GridSearchCV for Elastic Net...")
print(f"Testing λ values: {param_grid_elasticnet['elasticnet_regression__alpha']}")
print(f"Testing α (l1_ratio) values: {param_grid_elasticnet['elasticnet_regression__l1_ratio']}")
print(f"Total combinations: {len(param_grid_elasticnet['elasticnet_regression__alpha']) * len(param_grid_elasticnet['elasticnet_regression__l1_ratio'])}")
print("Using 5-fold cross-validation\n")

# Perform grid search
elasticnet_gscv = GridSearchCV(
    elasticnet_pipeline,
    param_grid_elasticnet,
    cv=5,
    scoring='r2',
    return_train_score=True,
    verbose=1
)

print("\nFitting GridSearchCV (this will take longer with 2 parameters)...")
elasticnet_gscv.fit(X, y)
print("\n✓ Grid search complete!")

## Elastic Net Tuning Results

In [ ]:
# Extract results
elasticnet_cv_results = pd.DataFrame(elasticnet_gscv.cv_results_)

print("\n" + "=" * 70)
print("ELASTIC NET TUNING RESULTS")
print("=" * 70)
print(f"\n✓ Best λ (alpha): {elasticnet_gscv.best_params_['elasticnet_regression__alpha']}")
print(f"✓ Best α (l1_ratio): {elasticnet_gscv.best_params_['elasticnet_regression__l1_ratio']}")
print(f"  Best CV R²: {elasticnet_gscv.best_score_:.4f}")

# Interpret the l1_ratio
l1_ratio = elasticnet_gscv.best_params_['elasticnet_regression__l1_ratio']
if l1_ratio < 0.3:
    interpretation = "More Ridge-like (emphasizes L2 penalty)"
elif l1_ratio > 0.7:
    interpretation = "More LASSO-like (emphasizes L1 penalty)"
else:
    interpretation = "Balanced mix of Ridge and LASSO"

print(f"\n  Interpretation: {interpretation}")
print(f"    L1 (LASSO) weight: {l1_ratio*100:.0f}%")
print(f"    L2 (Ridge) weight: {(1-l1_ratio)*100:.0f}%")
print("=" * 70)

# Fit best model and get coefficients
best_elasticnet = elasticnet_gscv.best_estimator_
elasticnet_coef = best_elasticnet.named_steps['elasticnet_regression'].coef_

# Count zero coefficients
n_zero_coef_en = np.sum(elasticnet_coef == 0)
n_nonzero_coef_en = np.sum(elasticnet_coef != 0)

print(f"\n🎯 FEATURE SELECTION (Elastic Net):")
print(f"  Total features: {len(elasticnet_coef)}")
print(f"  Non-zero coefficients: {n_nonzero_coef_en}")
print(f"  Zero coefficients (eliminated): {n_zero_coef_en}")
print(f"  Features kept: {(n_nonzero_coef_en/len(elasticnet_coef))*100:.1f}%")

## Visualization: Hyperparameter Grid

In [ ]:
# Create heatmap of hyperparameter combinations
# Reshape results into a grid
alphas = param_grid_elasticnet['elasticnet_regression__alpha']
l1_ratios = param_grid_elasticnet['elasticnet_regression__l1_ratio']

scores_grid = elasticnet_cv_results['mean_test_score'].values.reshape(
    len(alphas), len(l1_ratios)
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(scores_grid, cmap='RdYlGn', aspect='auto')

# Set ticks and labels
ax.set_xticks(np.arange(len(l1_ratios)))
ax.set_yticks(np.arange(len(alphas)))
ax.set_xticklabels([f"{r:.1f}" for r in l1_ratios])
ax.set_yticklabels([f"{a}" for a in alphas])

ax.set_xlabel('α (L1 Ratio)\n← More Ridge | More LASSO →', fontsize=12)
ax.set_ylabel('λ (Penalty Strength)\n← Weak | Strong →', fontsize=12)
ax.set_title('Elastic Net: Hyperparameter Grid Search Results\nCross-Validated R² Score', 
             fontsize=14, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('CV R² Score', rotation=270, labelpad=20)

# Add text annotations
for i in range(len(alphas)):
    for j in range(len(l1_ratios)):
        text = ax.text(j, i, f"{scores_grid[i, j]:.3f}",
                      ha="center", va="center", color="black", fontsize=9)

# Mark the best combination
best_alpha_idx = alphas.index(elasticnet_gscv.best_params_['elasticnet_regression__alpha'])
best_l1_idx = l1_ratios.index(elasticnet_gscv.best_params_['elasticnet_regression__l1_ratio'])
ax.add_patch(plt.Rectangle((best_l1_idx-0.5, best_alpha_idx-0.5), 1, 1, 
                           fill=False, edgecolor='blue', linewidth=3))

plt.tight_layout()
plt.show()

print("\n💡 The blue box shows the best hyperparameter combination!")

---

# 🏆 FINAL COMPARISON: All Models

In [ ]:
# Create comprehensive comparison
final_comparison = pd.DataFrame({
    'Model': [
        'Ordinary Linear Regression (OLS)',
        f"Ridge (λ={ridge_gscv.best_params_['ridge_regression__alpha']})",
        f"LASSO (λ={lasso_gscv.best_params_['lasso_regression__alpha']})",
        f"Elastic Net (λ={elasticnet_gscv.best_params_['elasticnet_regression__alpha']}, α={elasticnet_gscv.best_params_['elasticnet_regression__l1_ratio']})"
    ],
    'CV R²': [
        lr_cv_r2,
        ridge_gscv.best_score_,
        lasso_gscv.best_score_,
        elasticnet_gscv.best_score_
    ],
    'Non-Zero Coefs': [
        len(lr_coef),
        np.sum(ridge_coef_best != 0),
        n_nonzero_coef,
        n_nonzero_coef_en
    ],
    'Features Eliminated': [
        0,
        0,
        n_zero_coef,
        n_zero_coef_en
    ]
})

# Sort by CV R²
final_comparison = final_comparison.sort_values('CV R²', ascending=False)
final_comparison['Rank'] = range(1, len(final_comparison) + 1)
final_comparison = final_comparison[['Rank', 'Model', 'CV R²', 'Non-Zero Coefs', 'Features Eliminated']]

print("\n" + "=" * 90)
print("FINAL MODEL COMPARISON")
print("=" * 90)
print(final_comparison.to_string(index=False))
print("\n" + "=" * 90)
print(f"\n🏆 WINNER: {final_comparison.iloc[0]['Model']}")
print(f"   Cross-Validated R²: {final_comparison.iloc[0]['CV R²']:.4f}")
print(f"   Features Used: {final_comparison.iloc[0]['Non-Zero Coefs']}")
print(f"   Features Eliminated: {final_comparison.iloc[0]['Features Eliminated']}")
print("\n" + "=" * 90)

## Final Visualization: Model Performance

In [ ]:
# Create final comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Prepare data
models = final_comparison['Model'].tolist()
cv_r2 = final_comparison['CV R²'].tolist()
n_features = final_comparison['Non-Zero Coefs'].tolist()

colors = ['#2ecc71' if i == 0 else '#95a5a6' for i in range(len(models))]

# Plot 1: CV R² Comparison
bars1 = axes[0].barh(models, cv_r2, color=colors, edgecolor='black', linewidth=2)
axes[0].set_xlabel('Cross-Validated R² Score', fontsize=13, fontweight='bold')
axes[0].set_title('Model Performance\n🏆 Best Model in Green', 
                  fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Add value labels
for i, (bar, score) in enumerate(zip(bars1, cv_r2)):
    width = bar.get_width()
    if i == 0:
        axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {score:.4f} ★', ha='left', va='center', 
                    fontsize=12, fontweight='bold', color='#2ecc71')
    else:
        axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {score:.4f}', ha='left', va='center', fontsize=11)

# Plot 2: Feature Count Comparison
bars2 = axes[1].barh(models, n_features, color=colors, edgecolor='black', linewidth=2)
axes[1].set_xlabel('Number of Non-Zero Features', fontsize=13, fontweight='bold')
axes[1].set_title('Model Complexity\n(Fewer features = simpler model)', 
                  fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

# Add value labels
for i, (bar, count) in enumerate(zip(bars2, n_features)):
    width = bar.get_width()
    if i == 0:
        axes[1].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {count} ★', ha='left', va='center', 
                    fontsize=12, fontweight='bold', color='#2ecc71')
    else:
        axes[1].text(width, bar.get_y() + bar.get_height()/2, 
                    f'  {count}', ha='left', va='center', fontsize=11)

plt.tight_layout()
plt.show()

---

# 📋 FINAL ANSWER: Best Penalized Regression Model

In [ ]:
print("\n" + "=" * 80)
print("FINAL ANSWER - BEST PENALIZED REGRESSION MODEL")
print("=" * 80)

best_model_name = final_comparison.iloc[0]['Model']
best_r2 = final_comparison.iloc[0]['CV R²']
best_features = final_comparison.iloc[0]['Non-Zero Coefs']
eliminated = final_comparison.iloc[0]['Features Eliminated']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"\n   Cross-Validated R² Score: {best_r2:.4f}")
print(f"   Number of Features Used: {best_features}")
print(f"   Features Eliminated: {eliminated}")

print("\n" + "-" * 80)
print("WHY THIS MODEL?")
print("-" * 80)

if 'Ridge' in best_model_name:
    print("\nRidge Regression won because:")
    print("  ✓ Effectively shrinks coefficients to reduce overfitting")
    print("  ✓ Keeps all features but reduces their impact")
    print("  ✓ More stable than OLS with many correlated features")
    print("  ✓ Better cross-validated performance than other methods")
elif 'LASSO' in best_model_name:
    print("\nLASSO Regression won because:")
    print("  ✓ Performs automatic feature selection")
    print(f"  ✓ Eliminated {eliminated} unimportant features")
    print("  ✓ Creates a simpler, more interpretable model")
    print("  ✓ Better generalization through feature elimination")
elif 'Elastic Net' in best_model_name:
    print("\nElastic Net won because:")
    print("  ✓ Combines benefits of both Ridge and LASSO")
    print(f"  ✓ Eliminated {eliminated} features while shrinking others")
    print("  ✓ More flexible penalty structure")
    print("  ✓ Best balance between feature selection and shrinkage")
else:
    print("\nOLS won, which is surprising! This suggests:")
    print("  • The original features are well-chosen")
    print("  • Overfitting is not a major issue")
    print("  • Regularization may not be necessary")

# Compare to OLS
ols_r2 = final_comparison[final_comparison['Model'].str.contains('OLS')]['CV R²'].values[0]
improvement = best_r2 - ols_r2

print("\n" + "-" * 80)
print("IMPROVEMENT OVER BASELINE (OLS)")
print("-" * 80)
print(f"\n  OLS R²: {ols_r2:.4f}")
print(f"  Best Model R²: {best_r2:.4f}")
print(f"  Improvement: {improvement:.4f}")

if improvement > 0:
    if ols_r2 < 0:
        print(f"\n  🎉 HUGE IMPROVEMENT! OLS was severely overfitting (negative R²)")
        print(f"     Penalized regression fixed the overfitting problem!")
    else:
        pct_improvement = (improvement / abs(ols_r2)) * 100
        print(f"\n  ✓ {pct_improvement:.1f}% relative improvement")
        print(f"    Penalized regression successfully reduced overfitting!")
else:
    print(f"\n  ⚠ No improvement over OLS")
    print(f"    The dataset may not benefit much from regularization")

print("\n" + "=" * 80)
print("KEY TAKEAWAYS FROM CHAPTER 14")
print("=" * 80)
print("\n✓ Ridge Regression: Shrinks coefficients to reduce overfitting")
print("✓ LASSO: Performs automatic feature selection (zeros out coefficients)")
print("✓ Elastic Net: Combines both approaches for flexibility")
print("✓ Tuning hyperparameters (λ and α) is essential")
print("✓ Penalized regression helps when you have many features")
print("✓ Always use cross-validation to select the best model")
print("\n" + "=" * 80)

---

## 📊 Chapter 14 Complete Summary

### What We Learned:

1. **The Problem**: With many features, OLS can overfit the training data

2. **The Solution**: Add a penalty to the loss function

3. **Three Approaches**:
   - **Ridge**: $\ell(\beta) = SSE + \lambda \sum \beta_j^2$ (L2 penalty)
   - **LASSO**: $\ell(\beta) = SSE + \lambda \sum |\beta_j|$ (L1 penalty)
   - **Elastic Net**: $\ell(\beta) = SSE + \lambda[\alpha \sum |\beta_j| + (1-\alpha)\sum \beta_j^2]$ (both)

4. **Key Differences**:
   - Ridge shrinks coefficients but rarely zeros them out
   - LASSO can zero out coefficients (feature selection)
   - Elastic Net does both

5. **Hyperparameter Tuning**:
   - Always tune λ on a log scale
   - Use cross-validation to find optimal values
   - For Elastic Net, also tune the mixing parameter α

### When to Use Each Method:

- **Ridge**: When all features are potentially useful
- **LASSO**: When you want automatic feature selection
- **Elastic Net**: When you want the best of both worlds

---

## 🎯 Next Steps

1. Try different ranges for λ and α
2. Experiment with different feature engineering
3. Compare with other model types (Random Forest, Gradient Boosting)
4. Apply these techniques to your own datasets!